# Datová analýza exoplanetárního datasetu</h1>
## Semestrální práce z předmětu 4ST101
### *Autor: Jakub Vítovec*

## 1. Charakteristika zpracovávaných dat.
Ve své semestrální práci se budu zabývat daty získanými z veřejné databáze [Keggle](https://www.kaggle.com/). Konkrétní dataset je dostupný na URL [https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system](https://www.kaggle.com/datasets/sathyanarayanrao89/nasa-exoplanetary-system) a byl vytvořen uživatelem Sathyanarayan Rao z dat publikovaných vesmírnou agenturou NASA. Celkem se v datasetu nachází 32552 různých dosud objevených exoplanet a základní data o nich.
Exoplaneta je planeta, která obíhá okolo jiné hvězdy, než okolo Slunce. První exoplaneta byla objevena v roce 1998 a počet jejich počet stále roste. Ve své semestrální práci bych se rád zaměřil na trend růstu či poklesu počtu objevených nových exoplanet.

### Čištění dat
V tomto oddíle vyčistíme data od nežádoucích hodnot.

In [72]:
import numpy as np;
import pandas as pd;

dataset = pd.read_csv("./exoplanets_dataset.csv");
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32552 entries, 0 to 32551
Data columns (total 93 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unnamed: 0                               32552 non-null  int64  
 1   Planet Name                              32552 non-null  object 
 2   Host Name                                32552 non-null  object 
 3   Number of Stars                          32552 non-null  int64  
 4   Number of Planets                        32552 non-null  int64  
 5   Discovery Method                         32552 non-null  object 
 6   Discovery Year                           32552 non-null  int64  
 7   Discovery Facility                       32552 non-null  object 
 8   Orbital Period [days]                    29747 non-null  float64
 9    Planet Radius [Earth Radius]            22711 non-null  float64
 10  Planet Mass or Mass*sin(i) [Earth Mass]  4699 

Jak vidíme dataset obsahuje značné množství nepojmenovaných sloupců - Unnamed. Ty z datasetu odstraníme jako první.

In [73]:
for column_name in dataset.columns:
    if ("Unnamed" in column_name):
        del dataset[column_name];

Jako další by bylo vhodné zkontrolovat počty chybějících hodnot v jednotlivých sloupcích.

In [74]:
print(dataset.isnull().sum());

Planet Name                                    0
Host Name                                      0
Number of Stars                                0
Number of Planets                              0
Discovery Method                               0
Discovery Year                                 0
Discovery Facility                             0
Orbital Period [days]                       2805
 Planet Radius [Earth Radius]               9841
Planet Mass or Mass*sin(i) [Earth Mass]    27853
 Eccentricity                              15902
dtype: int64


Nejvíce chybějících hodnot se nachází ve sloupcích Planet Mass or Mass*sin(i), Eccentricity a Planet Radius, které obsahují hmotnost planety, ekcentricitu její oběžné dráhy a jejich poloměr. Vzhledem k tomu, že značná část těchto dat nám chybí, tak nebudeme tyto hodnoty využívat a sloupce odstraníme.

In [75]:
columns_to_delte = [" Planet Radius [Earth Radius]", "Planet Mass or Mass*sin(i) [Earth Mass]",  " Eccentricity"]

for column_name in dataset.columns:
    if (column_name in columns_to_delte):
        del dataset[column_name];

Jako další odstraníme z datasetu duplikované hodnoty.

In [76]:
print("Deleted duplicate values: " + str(dataset.duplicated().sum()));
dataset.drop_duplicates();

Deleted duplicate values: 11305


Po odstranění sloupců se značným počtem chybějících hodnot nám v datasetu zůstaly následující proměnné:
- **Planet Name** - Obsahuje jméno dané exoplanety
- **Host Name** - Jméno hostující hvězdy
- **Number of Stars** - Počer hvězd v planetárním systému dané exoplanety (u Země 1)
- **Number of Planets** - Počet planet v planetárním systému dané exoplanety (u Země 8)
- **Discovery Method** - Způsob objevení planety
- **Discovery Year** - Rok objevení planety
- **Discovary Facility** - Zařízení, které objevilo danou planetu
- **Orbital Period** - Orbitální perioda, doba v letech, za kterou planeta oběhne svou hostující hvězdu (Host Name)